聚类测试

In [1]:
# 数据处理
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Enable inline plotting
names = ['lat','lng','zero','alt','days','date','time']
streams = []
index = 0

userdata = 'D:\\cxcyProject\\Geolife Trajectories 1.3\\Data\\' + '002' + '\\Trajectory\\'
filelist = os.listdir(userdata)

for f in filelist:
    df_list = [pd.read_csv(userdata + f,header=6,names=names,index_col=False)]
    df = pd.concat(df_list, ignore_index=True)
    df.drop(['zero','alt','days','date','time'], axis=1, inplace=True)
    df_min = df.iloc[::12, :]
    lat_lng_data = np.c_[df_min['lat'].values, df_min['lng'].values]
    streams.append(lat_lng_data)

print(streams[0][0,0], streams[0][0,1])


39.927527 116.338899


In [2]:
# 分类
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.clustering import ResampleFeature
import numpy as np
from dipy.segment.clustering import QuickBundles
THRESHOLD = 2
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=256))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
        currD = np.mean(x)
        return currD


metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)

clusters = qb.cluster(streams)
print("Nb. clusters:", len(clusters))


Nb. clusters: 34


In [3]:
# 画图
from gmplot import gmplot
import random

def randomcolor():
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0,14)]
    return "#"+color

gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)

for clustersIndex in range(7):
    color = randomcolor()
    for i in clusters[clustersIndex].indices:
        gmap.plot(streams[i][:,0], streams[i][:,1], color, edge_width=1)

gmap.draw("user002_map.html")


In [ ]:
# 数据处理
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Enable inline plotting
names = ['lat','lng','zero','alt','days','date','time']
streams = []

base_path = 'D:\\cxcyProject\\Geolife Trajectories 1.3\\Data'

# 遍历所有用户（001到180）
for user_id in range(1, 100):
    # 格式化为三位数用户ID
    user_folder = f"{user_id:03d}"
    user_path = os.path.join(base_path, user_folder, 'Trajectory')
    
    # 检查路径是否存在
    if not os.path.exists(user_path):
        print(f"跳过不存在的路径: {user_path}")
        continue
    
    # 获取当前用户的所有轨迹文件
    filelist = os.listdir(user_path)
    
    # 处理每个轨迹文件
    for f in filelist:
        full_path = os.path.join(user_path, f)
        try:
            # 读取数据并处理
            df = pd.read_csv(full_path, header=6, names=names, index_col=False)
            df.drop(['zero','alt','days','date','time'], axis=1, inplace=True)
            # 每12行采样一次
            df_min = df.iloc[::25, :]
            lat_lng_data = np.c_[df_min['lat'].values, df_min['lng'].values]
            streams.append(lat_lng_data)
        except Exception as e:
            print(f"处理文件 {full_path} 时出错: {str(e)}")

# 打印测试数据（如果存在数据）
if streams and len(streams[0]) > 0:
    print(f"第一个轨迹点的坐标: {streams[0][0,0]}, {streams[0][0,1]}")
else:
    print("没有找到有效数据")

第一个轨迹点的坐标: 39.984198, 116.319322


In [2]:
# 数据处理
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random  # 新增随机模块

# Enable inline plotting
names = ['lat','lng','zero','alt','days','date','time']
streams = []

base_path = 'D:\\cxcyProject\\Geolife Trajectories 1.3\\Data'

# 随机选择30个用户（从001到180）
selected_users = random.sample(range(1, 181), 30)  # 生成30个不重复的随机数

for user_id in selected_users:
    # 格式化为三位数用户ID
    user_folder = f"{user_id:03d}"
    user_path = os.path.join(base_path, user_folder, 'Trajectory')
    
    # 检查路径是否存在
    if not os.path.exists(user_path):
        print(f"跳过不存在的路径: {user_path}")
        continue
    
    # 获取当前用户的所有轨迹文件
    try:
        filelist = os.listdir(user_path)
    except Exception as e:
        print(f"无法读取目录 {user_path}: {str(e)}")
        continue
    
    # 处理每个轨迹文件
    for filename in filelist:
        file_path = os.path.join(user_path, filename)
        try:
            # 读取数据并处理
            df = pd.read_csv(file_path, header=6, names=names, index_col=False)
            df.drop(['zero','alt','days','date','time'], axis=1, inplace=True)
            # 每20行采样一次
            df_min = df.iloc[::40, :]
            lat_lng_data = np.c_[df_min['lat'].values, df_min['lng'].values]
            streams.append(lat_lng_data)
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")

# 打印测试数据（如果存在数据）
if streams:
    print(f"第一个轨迹点的坐标: {streams[0][0,0]}, {streams[0][0,1]}")
    print(f"共处理了 {len(selected_users)} 个用户的 {len(streams)} 条轨迹")
else:
    print("没有找到有效数据")

第一个轨迹点的坐标: 39.9717, 116.313113333333
共处理了 30 个用户的 3319 条轨迹


In [4]:
# 分类
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.clustering import ResampleFeature
import numpy as np
from dipy.segment.clustering import QuickBundles
THRESHOLD = 2.5
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=256))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
        currD = np.mean(x)
        return currD


metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)

clusters = qb.cluster(streams)
print("Nb. clusters:", len(clusters))

Nb. clusters: 341


In [5]:
# 画图
from gmplot import gmplot
import random

def randomcolor():
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0,14)]
    return "#"+color

gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)

for clustersIndex in range(7):
    color = randomcolor()
    for i in clusters[clustersIndex].indices:
        gmap.plot(streams[i][:,0], streams[i][:,1], color, edge_width=1)

gmap.draw("user004_map.html")

In [ ]:
# 深度学习模块
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 原有模块
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.clustering import ResampleFeature, QuickBundles

# 超参数配置
THRESHOLD = 2.5
LATENT_DIM = 64  # 潜在空间维度
BATCH_SIZE = 32
EPOCHS = 50
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------------------------------
# 改进1：轨迹数据集类（含数据标准化）
# --------------------------------------------------
class TrajectoryDataset(Dataset):
    def __init__(self, streams):
        # 数据标准化：将经纬度归一化到[0,1]范围
        self.min_lat = min(traj[:,0].min() for traj in streams)
        self.max_lat = max(traj[:,0].max() for traj in streams)
        self.min_lng = min(traj[:,1].min() for traj in streams)
        self.max_lng = max(traj[:,1].max() for traj in streams)
        
        self.normalized = []
        for traj in streams:
            norm_traj = np.empty_like(traj)
            norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
            norm_traj[:,1] = (traj[:,1] - self.min_lng) / (self.max_lng - self.min_lng)
            self.normalized.append(norm_traj)
    
    def __len__(self):
        return len(self.normalized)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.normalized[idx].flatten()).to(DEVICE)

# --------------------------------------------------
# 改进2：时空自编码器模型
# --------------------------------------------------
class SpatioTemporalAE(nn.Module):
    def __init__(self, input_dim=256*2):
        super().__init__()
        # 编码器：捕捉时空模式
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, LATENT_DIM))
        
        # 解码器：重建轨迹
        self.decoder = nn.Sequential(
            nn.Linear(LATENT_DIM, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, input_dim))
        
    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

# --------------------------------------------------
# 改进3：深度学习增强的度量标准
# --------------------------------------------------
class DeepTrajectoryMetric(Metric):
    def __init__(self, encoder_model):
        super().__init__(feature=ResampleFeature(nb_points=256))
        self.encoder = encoder_model
        self.encoder.eval()  # 冻结模型
        
    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)
    
    def dist(self, v1, v2):
        with torch.no_grad():
            # 将numpy数组转换为张量
            tensor_v1 = torch.FloatTensor(v1.flatten()).unsqueeze(0).to(DEVICE)
            tensor_v2 = torch.FloatTensor(v2.flatten()).unsqueeze(0).to(DEVICE)
            
            # 提取潜在特征
            z1 = self.encoder(tensor_v1)
            z2 = self.encoder(tensor_v2)
            
            # 计算余弦相似度
            return 1 - torch.cosine_similarity(z1, z2).item()

# --------------------------------------------------
# 主流程修改
# --------------------------------------------------
if __name__ == "__main__":
    # ========== 数据加载 ==========
    # （原始数据加载部分保持不变）
    # ... [原有数据加载代码] ...
    
    # ========== 数据预处理 ==========
    # 确保所有轨迹统一长度
    resampler = ResampleFeature(nb_points=256)
    streams = [resampler.extract(traj) for traj in streams]
    
    # 创建数据集
    dataset = TrajectoryDataset(streams)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # ========== 模型训练 ==========
    model = SpatioTemporalAE().to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    # 训练循环
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            _, recon = model(batch)
            loss = criterion(recon, batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(dataloader):.4f}")
    
    # ========== 深度聚类 ==========
    # 创建深度学习度量标准
    metric = DeepTrajectoryMetric(model.encoder)
    
    # 初始化QuickBundles
    qb = QuickBundles(threshold=THRESHOLD, metric=metric)
    
    # 执行聚类（使用原始streams，因Resample已在预处理完成）
    clusters = qb.cluster(streams)
    print("Nb. clusters:", len(clusters))
    
    # ========== 可视化（可选） ==========

C:\Users\ZYPC\AppData\Local\Temp\ipykernel_24996\1568629575.py:38: RuntimeWarning: overflow encountered in scalar subtract
  norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
C:\Users\ZYPC\AppData\Local\Temp\ipykernel_24996\1568629575.py:38: RuntimeWarning: overflow encountered in subtract
  norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
C:\Users\ZYPC\AppData\Local\Temp\ipykernel_24996\1568629575.py:38: RuntimeWarning: invalid value encountered in divide
  norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
C:\Users\ZYPC\AppData\Local\Temp\ipykernel_24996\1568629575.py:38: RuntimeWarning: invalid value encountered in subtract
  norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)


Epoch 1/50, Loss: nan
Epoch 2/50, Loss: nan
Epoch 3/50, Loss: nan
Epoch 4/50, Loss: nan
Epoch 5/50, Loss: nan
Epoch 6/50, Loss: nan
Epoch 7/50, Loss: nan
Epoch 8/50, Loss: nan
Epoch 9/50, Loss: nan
Epoch 10/50, Loss: nan
Epoch 11/50, Loss: nan
Epoch 12/50, Loss: nan
Epoch 13/50, Loss: nan
Epoch 14/50, Loss: nan
Epoch 15/50, Loss: nan
Epoch 16/50, Loss: nan
Epoch 17/50, Loss: nan
Epoch 18/50, Loss: nan
Epoch 19/50, Loss: nan
Epoch 20/50, Loss: nan
Epoch 21/50, Loss: nan
Epoch 22/50, Loss: nan
Epoch 23/50, Loss: nan
Epoch 24/50, Loss: nan
Epoch 25/50, Loss: nan
Epoch 26/50, Loss: nan
Epoch 27/50, Loss: nan
Epoch 28/50, Loss: nan
Epoch 29/50, Loss: nan
Epoch 30/50, Loss: nan
Epoch 31/50, Loss: nan
Epoch 32/50, Loss: nan
Epoch 33/50, Loss: nan
Epoch 34/50, Loss: nan
Epoch 35/50, Loss: nan
Epoch 36/50, Loss: nan
Epoch 37/50, Loss: nan
Epoch 38/50, Loss: nan
Epoch 39/50, Loss: nan
Epoch 40/50, Loss: nan
Epoch 41/50, Loss: nan
Epoch 42/50, Loss: nan
Epoch 43/50, Loss: nan
Epoch 44/50, Loss: n

KeyboardInterrupt: 

Exception ignored in: 'dipy.segment.clusteringspeed.QuickBundles.find_nearest_cluster'
Traceback (most recent call last):
  File "dipy/segment/metricspeed.pyx", line 59, in dipy.segment.metricspeed.Metric.c_dist
  File "dipy/segment/metricspeed.pyx", line 81, in dipy.segment.metricspeed.Metric.dist
  File "C:\Users\ZYPC\AppData\Local\Temp\ipykernel_24996\1568629575.py", line 98, in dist
  File "d:\Programs Files\anaconda3\Lib\site-packages\torch\nn\modules\module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programs Files\anaconda3\Lib\site-packages\torch\nn\modules\module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programs Files\anaconda3\Lib\site-packages\torch\nn\modules\container.py", line 250, in forward
    input = module(input)
            ^^^^^^^^^^^^^
  File "d:\Programs Files\anaconda3\Lib\site-packages\tor

In [1]:
# 数据处理
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random  # 新增随机模块

# Enable inline plotting
names = ['lat','lng','zero','alt','days','date','time']
streams = []

base_path = 'D:\\cxcyProject\\Geolife Trajectories 1.3\\Data'

# 随机选择30个用户（从001到180）
selected_users = random.sample(range(1, 181), 3)  # 生成30个不重复的随机数

for user_id in selected_users:
    # 格式化为三位数用户ID
    user_folder = f"{user_id:03d}"
    user_path = os.path.join(base_path, user_folder, 'Trajectory')
    
    # 检查路径是否存在
    if not os.path.exists(user_path):
        print(f"跳过不存在的路径: {user_path}")
        continue
    
    # 获取当前用户的所有轨迹文件
    try:
        filelist = os.listdir(user_path)
    except Exception as e:
        print(f"无法读取目录 {user_path}: {str(e)}")
        continue
    
    # 处理每个轨迹文件
    for filename in filelist:
        file_path = os.path.join(user_path, filename)
        try:
            # 读取数据并处理
            df = pd.read_csv(file_path, header=6, names=names, index_col=False)
            df.drop(['zero','alt','days','date','time'], axis=1, inplace=True)
            # 每20行采样一次
            df_min = df.iloc[::20, :]
            lat_lng_data = np.c_[df_min['lat'].values, df_min['lng'].values]
            streams.append(lat_lng_data)
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")

# 打印测试数据（如果存在数据）
if streams:
    print(f"第一个轨迹点的坐标: {streams[0][0,0]}, {streams[0][0,1]}")
    print(f"共处理了 {len(selected_users)} 个用户的 {len(streams)} 条轨迹")
else:
    print("没有找到有效数据")

第一个轨迹点的坐标: 26.161528, 119.943234
共处理了 3 个用户的 72 条轨迹


In [3]:
# 数据处理部分保持不变
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random

# 新增深度学习依赖（放在传统方法导入之前）
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 传统方法模块导入（关键修正点）
from dipy.segment.metric import Metric  # 确保在自定义类之前导入
from dipy.segment.clustering import ResampleFeature, QuickBundles
import geopy.distance
THRESHOLD = 1.5
# 可视化模块
from gmplot import gmplot


# 深度学习数据预处理类
class TrajectoryDataset(Dataset):
    def __init__(self, streams):
        # 保持原有归一化逻辑
        self.min_lat = min(traj[:,0].min() for traj in streams)
        self.max_lat = max(traj[:,0].max() for traj in streams)
        self.min_lng = min(traj[:,1].min() for traj in streams)
        self.max_lng = max(traj[:,1].max() for traj in streams)
        
        self.normalized = []
        for traj in streams:
            norm_traj = np.empty_like(traj)
            norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
            norm_traj[:,1] = (traj[:,1] - self.min_lng) / (self.max_lng - self.min_lng)
            self.normalized.append(norm_traj.flatten())
    
    def __len__(self):
        return len(self.normalized)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.normalized[idx])

# 深度学习模型（保持原有结构）
class TrajectoryAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512)
        )
        
    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

# 修正后的距离度量类
class DeepTrajectoryMetric(Metric):  # 确保继承自dipy的Metric
    def __init__(self, encoder):
        super().__init__(feature=ResampleFeature(nb_points=256))  # 显式调用父类初始化
        self.encoder = encoder
        self.encoder.eval()
        
    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)
    
    def dist(self, v1, v2):
        with torch.no_grad():
            tensor_v1 = torch.FloatTensor(v1.flatten()).unsqueeze(0)
            tensor_v2 = torch.FloatTensor(v2.flatten()).unsqueeze(0)
            
            z1 = self.encoder(tensor_v1)
            z2 = self.encoder(tensor_v2)
            
            return 1 - torch.cosine_similarity(z1, z2).item()

# 主流程整合
if __name__ == "__main__":
    # ===== 数据加载 =====
    # ... [原有数据加载代码] ...
    
    # ===== 数据预处理 =====
    # 统一轨迹长度（关键衔接点）
    resampler = ResampleFeature(nb_points=256)
    streams = [resampler.extract(traj) for traj in streams]
    
    # ===== 深度学习训练 =====
    dataset = TrajectoryDataset(streams)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    model = TrajectoryAE()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(50):
        total_loss = 0
        for batch in dataloader:
            _, recon = model(batch)
            loss = criterion(recon, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")
    
    # ===== 深度聚类 =====
    metric = DeepTrajectoryMetric(model.encoder)
    qb = QuickBundles(threshold=THRESHOLD, metric=metric)
    clusters = qb.cluster(streams)
    print("Nb. clusters:", len(clusters))
    
    # ===== 可视化 =====
    def randomcolor():
        colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
        return "#" + ''.join(random.choice(colorArr) for _ in range(6))
    
    gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)
    
    for cluster_id in range(min(7, len(clusters))):  # 防止集群数不足7
        color = randomcolor()
        for i in clusters[cluster_id].indices:
            # 使用原始经纬度数据绘制
            lat_points = streams[i][:,0]
            lng_points = streams[i][:,1]
            gmap.plot(lat_points, lng_points, color, edge_width=1)
    
    gmap.draw("deep_clusters_map2.html")
  

Epoch 1, Loss: 0.4855
Epoch 2, Loss: 0.4218
Epoch 3, Loss: 0.2345
Epoch 4, Loss: 0.1005
Epoch 5, Loss: 0.0311
Epoch 6, Loss: 0.0317
Epoch 7, Loss: 0.0259
Epoch 8, Loss: 0.0216
Epoch 9, Loss: 0.0177
Epoch 10, Loss: 0.0100
Epoch 11, Loss: 0.0090
Epoch 12, Loss: 0.0067
Epoch 13, Loss: 0.0062
Epoch 14, Loss: 0.0040
Epoch 15, Loss: 0.0044
Epoch 16, Loss: 0.0031
Epoch 17, Loss: 0.0033
Epoch 18, Loss: 0.0027
Epoch 19, Loss: 0.0028
Epoch 20, Loss: 0.0025
Epoch 21, Loss: 0.0076
Epoch 22, Loss: 0.0024
Epoch 23, Loss: 0.0086
Epoch 24, Loss: 0.0037
Epoch 25, Loss: 0.0022
Epoch 26, Loss: 0.0022
Epoch 27, Loss: 0.0022
Epoch 28, Loss: 0.0022
Epoch 29, Loss: 0.0022
Epoch 30, Loss: 0.0072
Epoch 31, Loss: 0.0023
Epoch 32, Loss: 0.0023
Epoch 33, Loss: 0.0035
Epoch 34, Loss: 0.0022
Epoch 35, Loss: 0.0022
Epoch 36, Loss: 0.0022
Epoch 37, Loss: 0.0021
Epoch 38, Loss: 0.0022
Epoch 39, Loss: 0.0021
Epoch 40, Loss: 0.0035
Epoch 41, Loss: 0.0021
Epoch 42, Loss: 0.0021
Epoch 43, Loss: 0.0021
Epoch 44, Loss: 0.00

In [2]:
  
   # 数据处理部分保持不变
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random

# 新增深度学习依赖（放在传统方法导入之前）
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 传统方法模块导入（关键修正点）
from dipy.segment.metric import Metric  # 确保在自定义类之前导入
from dipy.segment.clustering import ResampleFeature, QuickBundles
import geopy.distance
THRESHOLD = 1.5
# 可视化模块
from gmplot import gmplot


# 深度学习数据预处理类
class TrajectoryDataset(Dataset):
    def __init__(self, streams):
        # 保持原有归一化逻辑
        self.min_lat = min(traj[:,0].min() for traj in streams)
        self.max_lat = max(traj[:,0].max() for traj in streams)
        self.min_lng = min(traj[:,1].min() for traj in streams)
        self.max_lng = max(traj[:,1].max() for traj in streams)
        
        self.normalized = []
        for traj in streams:
            norm_traj = np.empty_like(traj)
            norm_traj[:,0] = (traj[:,0] - self.min_lat) / (self.max_lat - self.min_lat)
            norm_traj[:,1] = (traj[:,1] - self.min_lng) / (self.max_lng - self.min_lng)
            self.normalized.append(norm_traj.flatten())
    
    def __len__(self):
        return len(self.normalized)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.normalized[idx])

# 深度学习模型（保持原有结构）
class TrajectoryAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512)
        )
        
    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

# 新增注意力机制模块
class SpatioTemporalAttention(nn.Module):
    def __init__(self, input_dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = input_dim // num_heads
        
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape  # [batch, 256, 2]
        
        # 多头注意力计算
        Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # 注意力得分
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))
        attn_weights = self.softmax(scores)
        
        # 上下文向量
        context = torch.matmul(attn_weights, V).transpose(1, 2).contiguous()
        context = context.view(batch_size, seq_len, -1)
        return context

# 改进的深度学习模型
class AttnTrajectoryAE(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 编码器
        self.encoder = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Unflatten(1, (256, 2)),  # 恢复时空结构 [batch, 256, 2]
            SpatioTemporalAttention(input_dim=2),
            nn.Flatten(),
            nn.Linear(256*2, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )
        
        # 解码器
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256*2),
            nn.Unflatten(1, (256, 2)),
            SpatioTemporalAttention(input_dim=2),
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # 编码阶段
        x_reshaped = self.encoder[:3](x)  # [batch, 256, 2]
        attn_output = self.encoder[3](x_reshaped)
        z = self.encoder[4:](attn_output)
        
        # 解码阶段
        recon = self.decoder(z)
        return z, recon

# 修改模型初始化部分
if __name__ == "__main__":
    # ... [数据加载和预处理部分不变] ...
    
    # 使用新模型
    model = AttnTrajectoryAE()
    
    # 修改优化器参数
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
# 修正后的距离度量类
class DeepTrajectoryMetric(Metric):  # 确保继承自dipy的Metric
    def __init__(self, encoder):
        super().__init__(feature=ResampleFeature(nb_points=256))  # 显式调用父类初始化
        self.encoder = encoder
        self.encoder.eval()
        
    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)
    
    def dist(self, v1, v2):
        with torch.no_grad():
            tensor_v1 = torch.FloatTensor(v1.flatten()).unsqueeze(0)
            tensor_v2 = torch.FloatTensor(v2.flatten()).unsqueeze(0)
            
            z1 = self.encoder(tensor_v1)
            z2 = self.encoder(tensor_v2)
            
            return 1 - torch.cosine_similarity(z1, z2).item()

# 主流程整合
if __name__ == "__main__":
    # ===== 数据加载 =====
    # ... [原有数据加载代码] ...
    
    # ===== 数据预处理 =====
    # 统一轨迹长度（关键衔接点）
    resampler = ResampleFeature(nb_points=256)
    streams = [resampler.extract(traj) for traj in streams]
    
    # ===== 深度学习训练 =====
    dataset = TrajectoryDataset(streams)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    model = TrajectoryAE()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(50):
        total_loss = 0
        for batch in dataloader:
            _, recon = model(batch)
            loss = criterion(recon, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")
    
    # ===== 深度聚类 =====
    metric = DeepTrajectoryMetric(model.encoder)
    qb = QuickBundles(threshold=THRESHOLD, metric=metric)
    clusters = qb.cluster(streams)
    print("Nb. clusters:", len(clusters))
    
    # ===== 可视化 =====
    def randomcolor():
        colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
        return "#" + ''.join(random.choice(colorArr) for _ in range(6))
    
    gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)
    
    for cluster_id in range(min(7, len(clusters))):  # 防止集群数不足7
        color = randomcolor()
        for i in clusters[cluster_id].indices:
            # 使用原始经纬度数据绘制
            lat_points = streams[i][:,0]
            lng_points = streams[i][:,1]
            gmap.plot(lat_points, lng_points, color, edge_width=1)
    
    gmap.draw("deep_clusters_map4.html")

Epoch 1, Loss: 0.4856
Epoch 2, Loss: 0.4058
Epoch 3, Loss: 0.1899
Epoch 4, Loss: 0.0998
Epoch 5, Loss: 0.0209
Epoch 6, Loss: 0.0324
Epoch 7, Loss: 0.0176
Epoch 8, Loss: 0.0233
Epoch 9, Loss: 0.0107
Epoch 10, Loss: 0.0099
Epoch 11, Loss: 0.0040
Epoch 12, Loss: 0.0045
Epoch 13, Loss: 0.0029
Epoch 14, Loss: 0.0028
Epoch 15, Loss: 0.0014
Epoch 16, Loss: 0.0015
Epoch 17, Loss: 0.0009
Epoch 18, Loss: 0.0008
Epoch 19, Loss: 0.0006
Epoch 20, Loss: 0.0005
Epoch 21, Loss: 0.0005
Epoch 22, Loss: 0.0003
Epoch 23, Loss: 0.0004
Epoch 24, Loss: 0.0007
Epoch 25, Loss: 0.0002
Epoch 26, Loss: 0.0002
Epoch 27, Loss: 0.0002
Epoch 28, Loss: 0.0001
Epoch 29, Loss: 0.0001
Epoch 30, Loss: 0.0001
Epoch 31, Loss: 0.0001
Epoch 32, Loss: 0.0001
Epoch 33, Loss: 0.0001
Epoch 34, Loss: 0.0001
Epoch 35, Loss: 0.0001
Epoch 36, Loss: 0.0001
Epoch 37, Loss: 0.0002
Epoch 38, Loss: 0.0001
Epoch 39, Loss: 0.0001
Epoch 40, Loss: 0.0001
Epoch 41, Loss: 0.0001
Epoch 42, Loss: 0.0001
Epoch 43, Loss: 0.0000
Epoch 44, Loss: 0.00